# Predict delays for every scheduled connection

In this notebook, we provide schedule connections with an uncertainty parameter for prediciting possible delays. In other words, we map the static public transport network with the delay analysis in order to obtain a meaningful and accurate predictive model. The notebook is structured as follows: 

*   **[Start Spark](#spark)** 
*   **[Merge connections and probabilities](#merge)**  
*   **[Get predictive model](#predictive)** 
  



<a id = 'spark'></a>
### 1. Start Spark

We will be using a Spark Session for performing different transformations and actions on dataframes

In [ ]:
%%configure -f
{
    "conf": {
        "spark.app.name": "datavirus_delay_prediction",
    }
}

In [2]:
spark

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import pyspark.sql.functions as f
from pyspark.sql import Window

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<a id = 'merge'></a>
### 2. Merge connections and probabilities

We load our two dataframes: the "ideal" schedule for the connections and the one containing the delay probabilities. We are able to join them thanks to the previously performed transformations. However, we have to be careful, there is not a perfect 1-to-1 correspondence between the connections data and the probability information. There is no probability data available for some of the connections. For those connections we use the transport probability data which we created in the [delay_analysis](./delay_analysis.ipynb) notebook.

In [15]:
id_connections = spark.read.orc('/user/datavirus/id_connections.orc').repartition(150, 'id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
probability = spark.read.orc('/user/datavirus/probability.orc').repartition(150, 'id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# join the two dataframes
probability_connections = (
    id_connections
    .join(probability, ['id', 'station_id', 'arrival_time_minute'], how='left_outer')
    .select(
        id_connections.stop_sequence,
        id_connections.route_type,
        id_connections.arrival_time_hour,
        id_connections.produkt_id,
        id_connections.start_id,
        id_connections.start_time,
        id_connections.trip_id,
        probability.transport_type,
        probability.line_text,
        id_connections.stop_time,
        id_connections.stop_id,
        probability.delay_probability,
        probability.delay_parameter
    )
)

probability_connections.write.format('orc').save('/user/datavirus/probability_connections.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# "back-up" data for trips without corresponding
# probability data
transport_probability = spark.read.orc('/user/datavirus/transport_probability.orc').alias('transport_probability')
transport_probability = (
    transport_probability
    .withColumn('arrival_time_hour', transport_probability.ankunftszeit_hour)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<a id = 'predictive'></a>
### 3. Get predictive model

At last, we can obtain the final connection dataframe with the probability parameters included. This will help us predict the future delay of any connection incorporated in our transport network

In [27]:
# add transport_probability information
# for missing data
connections = (
    probability_connections
    .join(transport_probability, ['produkt_id', 'arrival_time_hour'])
    .select(
        probability_connections.stop_sequence,
        probability_connections.route_type,
        probability_connections.start_id,
        probability_connections.start_time,
        probability_connections.trip_id,
        probability_connections.produkt_id.alias('transport_type'),
        probability_connections.line_text,
        probability_connections.stop_time,
        probability_connections.stop_id,
        f.when(
            f.col('probability_connections.delay_probability').isNotNull(),
            f.col('probability_connections.delay_probability')
        ).otherwise(
            f.col('transport_probability.transport_delay_probability')
        ).alias('delay_probability'),
        f.when(f.col('probability_connections.delay_parameter').isNotNull(),
              f.col('probability_connections.delay_parameter')
        ).otherwise(
            f.col('transport_probability.transport_delay_parameter')
        ).alias('delay_parameter')
    )
    .orderBy([probability_connections.stop_time.desc(), probability_connections.stop_sequence.desc()])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
connections.write.format('csv').save('/user/datavirus/connections.csv', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
connections.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+--------+----------+-----------------------+--------------+---------+---------+-------+------------------+--------------------+
|stop_sequence|route_type|start_id|start_time|trip_id                |transport_type|line_text|stop_time|stop_id|delay_probability |delay_parameter     |
+-------------+----------+--------+----------+-----------------------+--------------+---------+---------+-------+------------------+--------------------+
|2            |700       |8502209 |10:05:00  |9.TA.30-170-Y-j19-1.1.H|bus           |null     |10:05:00 |8502209|0.9193866847990115|0.011589134124321832|
|2            |700       |8502771 |10:05:00  |392.TA.26-235-j19-1.5.R|bus           |235      |10:05:00 |8502771|0.9159136766383016|0.013732196842403623|
|2            |700       |8502771 |10:05:00  |493.TA.26-235-j19-1.5.R|bus           |235      |10:05:00 |8502771|0.9159136766383016|0.013732196842403623|
|2            |700       |8502771 |10:05:00  |360.TA.26-235-j19-1.5.R|bus   